In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm 
from pathlib import Path
import pandas as pd
import einops
from sklearn.metrics.pairwise import cosine_distances
import torch as t 
from torch import Tensor, from_numpy
import torch.nn.functional as F
from torchviz import make_dot
%matplotlib

### take patches

In [ ]:
from utils import generatePatches

In [ ]:
data_path = '/Users/thomasbush/Documents/DSS_Tilburg/data/keyframes/_2025-04-22 00:25:47.432414_keyframes.pth'
data = t.load(data_path, map_location='cpu', weights_only=False)
keyframes = data['keyframes']
idx = data['keyframe_idx']

In [ ]:
# quick script to display 8 keyframes 
idxs = t.randperm(keyframes.shape[0])[:8]
selected_keyframes = keyframes[idxs]

# Plot them in a 2x4 grid
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
for i, ax in enumerate(axes.flat):
    ax.imshow(selected_keyframes[i].cpu().numpy(), cmap='gray')
    ax.set_title(f"Keyframe {idxs[i].item()}")
    ax.axis('off')

plt.tight_layout()
# plt.savefig("selected_keyframes_grid.png", dpi=300)  # Save if you want
plt.show()

In [ ]:
for i in idxs:
    plt.figure()
    plt.imshow(keyframes[i], cmap='gray')
    plt.axis('Off')
    plt.savefig(save_path / f"keyframe{i}.svg", format="svg", bbox_inches="tight")


In [ ]:
img = keyframes[100]
plt.figure()
plt.imshow(img, cmap='gray')
# plt.title('Original Frame')
plt.axis('Off')
plt.savefig(save_path / "origina_frame.svg", format="svg", bbox_inches="tight")
plt.show()

In [ ]:
patches_tensor, original_shape, padding  = generatePatches(img.unsqueeze(dim=0), dim=64)

In [ ]:
patches_tensor = patches_tensor.squeeze()

In [ ]:
import matplotlib.patches as patches

In [ ]:
def plot_image_with_patches(patches_tensor, original_shape, patch_dim, padding, save_path: None|Path):
    """
    patches: tensor of shape (N, d, d) or (N, 1, d, d)
    original_shape: tuple (H, W)
    patch_dim: int
    padding: tuple (top, bottom, left, right)
    """
    H, W = original_shape
    pad_top, pad_bottom, pad_left, pad_right = padding

    # Total padded dimensions
    H_p = H + pad_top + pad_bottom
    W_p = W + pad_left + pad_right

    # Convert patches to tensor if needed
    if isinstance(patches_tensor, np.ndarray):
        patches_tensor = t.tensor(patches_tensor)

    # If patches have channels, remove it
    if patches_tensor.dim() == 4 and patches_tensor.shape[1] == 1:
        patches_tensor = patches_tensor.squeeze(1)

    # Reconstruct the image from patches
    num_patches_h = H_p // patch_dim
    num_patches_w = W_p // patch_dim
    assert patches_tensor.shape[0] == num_patches_h * num_patches_w

    patches_grid = patches_tensor.view(num_patches_h, num_patches_w, patch_dim, patch_dim)
    reconstructed = patches_grid.permute(0, 2, 1, 3).contiguous().view(H_p, W_p)

    # Remove the padding to recover the original image
    reconstructed = reconstructed[pad_top:pad_top+H, pad_left:pad_left+W]

    # Plot the image
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.imshow(reconstructed, cmap='gray')
    # ax.set_title("Image with Patch Boundaries")

    for i in range(0, H_p, patch_dim):
        for j in range(0, W_p, patch_dim):
            # Draw rectangles in padded space
            y = i - pad_top
            x = j - pad_left
            rect = patches.Rectangle((x, y), patch_dim, patch_dim,
                                            linewidth=0.5, edgecolor='blue', facecolor='none')
            ax.add_patch(rect)

    plt.axis('off')
    plt.tight_layout()
    if save_path is not None:
        plt.savefig(save_path / "patches_overlay.svg", format="svg", bbox_inches="tight")
    plt.show()

In [ ]:
save_path = Path('/Users/thomasbush/Documents/DSS_Tilburg/data/images')
plot_image_with_patches(patches_tensor, (original_shape[1], original_shape[2]), 64, padding, save_path)

In [ ]:
save_path = Path('/Users/thomasbush/Documents/DSS_Tilburg/data/images')
plt.savefig(save_path / "patches_overlay.svg", format="svg", bbox_inches="tight")

In [ ]:
plt.figure()
plt.imshow(patches_tensor[24], cmap='gray')
plt.axis('Off')
plt.savefig(save_path / "patch.svg", format="svg", bbox_inches="tight")

plt.show()

In [ ]:
img_n = einops.rearrange(patches_tensor[24], 'h w -> 1 1 h w')

In [ ]:
def applyInverseCrf(img: t.Tensor, gamma: float = 2.2) -> t.Tensor:
    return img ** gamma
def addRealisticNoise(img: t.Tensor, sigma_s=0.12, sigma_c=0.03) -> t.Tensor:
    """
    Realistic noise: L + n_s + n_c
    img: (B, C, H, W), float32 in [0,1]
    """
    # Inverse CRF (simulate raw sensor signal)
    linear_img = applyInverseCrf(img)

    # Signal-dependent noise (shot noise)
    noise_s = t.randn_like(linear_img) * t.sqrt(t.clamp(linear_img * sigma_s**2, min=1e-6))

    # Constant noise (read noise)``
    noise_c = t.randn_like(linear_img) * sigma_c

    noisy_img = linear_img + noise_s + noise_c
    noisy_img = t.clamp(noisy_img, 0, 1)
    return (noisy_img ** (1 / 2.2)).clamp(0, 1) * 1.5  # increase brightness/contrast


img_noisy = addRealisticNoise(img_n, sigma_s=0.3, sigma_c=0.1)

In [ ]:
plt.figure()
plt.imshow(img_noisy.squeeze(dim=(0, 1)), cmap='gray')
plt.axis('Off')
plt.show()

### others

In [ ]:
import sys
sys.path.append('..')  # Add parent directory to Python path
from utils import loadVideoArray, splitPadding
from denoising.models import AutoEncoder

In [ ]:
model = AutoEncoder(latent_dim_size=32, hidden_dim_size=128)
x = t.randn((5, 1, 64, 64))

y = model.forward(x)
make_dot(y.mean(), params=dict(model.encoder.named_parameters()))

### Loading Data:

In [ ]:
video_path = "/Users/thomasbush/Documents/Vault/Iurilli_lab/3d_tracking/data/multicam_video_2024-07-22T10_19_22_cropped_20250325101012/multicam_video_2024-07-22T10_19_22_mirror-left.avi.mp4"

In [ ]:
import sys
sys.path.append('..')  # Add parent directory to Python path
from utils import loadVideoArray, splitPadding

In [ ]:
vid = loadVideoArray(video_path)

In [ ]:
vid.shape

In [ ]:
idxs = np.linspace(0, vid.shape[0]-1, 4).astype(int)
frames = vid[idxs]

In [ ]:

fig, axes = plt.subplots(2, 2, figsize=(16, 8))
for i, ax in enumerate(axes.flat):
    ax.imshow(frames[i], cmap='gray')
    ax.axis('off')

plt.tight_layout()
plt.savefig(save_path / "frames_ex.png", dpi=300)  # Save if you want
plt.show()

In [ ]:
def getFirstFrame(videofile):
    vidcap = cv2.VideoCapture(videofile)
    success, image = vidcap.read()
    # if success:
    #     cv2.imwrite("first_frame.jpg", image)
    return image
full_path = '/Users/thomasbush/Documents/Vault/Iurilli_lab/3d_tracking/data/uncropped_cal/multicam_video_2024-07-24T14_13_45.avi'

first_frame = getFirstFrame(full_path)

In [ ]:
plt.figure()
plt.imshow(first_frame, cmap='gray')
plt.axis('Off')
plt.savefig(save_path / 'fullimg.png', dpi=300)
plt.show()

In [ ]:
avg_brightness = vid.mean(axis=(1,2))  # assuming shape (T, H, W)
plt.plot(avg_brightness)

In [ ]:
from skimage.measure import shannon_entropy
entropies = [shannon_entropy(f) for f in vid]

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(entropies)
plt.xlabel("Frame Index")
plt.ylabel("Shannon Entropy")
plt.title("Frame Entropy Over Time")
plt.grid(True)
plt.tight_layout()
plt.savefig(save_path / 'entropyvid.png', dpi=300)
plt.show()

In [ ]:
plt.figure(figsize=(6, 4))
plt.hist(vid.flatten(), bins=100, color='gray')
plt.xlabel("Pixel Intensity")
plt.ylabel("Frequency")
plt.title("Pixel Intensity Distribution (All Frames)")
plt.tight_layout()
plt.savefig(save_path / 'pixelinteistdist.png', dpi=300)
plt.show()

### Method 1: Frame Difference-Based Keyframe Selection

Key frames are frames that represent transitons or new visual content in a video sequence. Thus, we are going to try to select frames where the pixel-level difference is the greatest

We have decided to use a Histogram Difference (statistical change) that uses the comparison between distribution of pixel intensities:

1. Convert img to hist
2. compute hist diff: like L1 or cos sim
3. Select the top-k frames with the highest score

In [ ]:
# we know compute the hist 
def compute_histograms(frames: np.ndarray, bins: int = 64) -> np.ndarray:
    n_frames = frames.shape[0]
    
    flat_vid = einops.rearrange(frames, 'frame h w -> frame (h w)')

    histograms = []
    for i in tqdm(range(n_frames), desc='Compute pixel distribution'):
        hist, _ = np.histogram(flat_vid[i], bins=bins, range=(0, 255), density=True)
        histograms.append(hist)

    histograms = np.stack(histograms)  # shape: (n_frames, bins)
    assert histograms.shape == (n_frames, bins)
    return histograms

In [ ]:
hist = compute_histograms(video_array)

In [ ]:
# compute the diff 
def compute_cosine_diff(histograms: np.ndarray) -> np.ndarray:
    # Normalize the histograms first (L2 norm)
    norms = np.linalg.norm(histograms, axis=1, keepdims=True)
    hist_norm = histograms / (norms + 1e-8)  # prevent divide by 0

    # Shift histograms to compare frame t and t-1
    h1 = hist_norm[1:]       # t = 1 to N-1
    h0 = hist_norm[:-1]      # t = 0 to N-2

    # Cosine similarity = dot product of normalized vectors
    sim = np.sum(h1 * h0, axis=1)  # shape: (n_frames - 1,)
    diff = 1 - sim  # cosine distance (1 = completely different)

    return diff  # shape: (n_frames - 1,)

In [ ]:
cos_sim = compute_cosine_diff(hist)

In [ ]:
def select_keyframes(differences:np.ndarray, k:int = 30):
    idx = np.argsort(differences)[-k:]
    return np.sort(idx+1)

In [ ]:
idx = select_keyframes(cos_sim)
keyframes = video_array[idx, ...]

### Method 2: Features Extraction 

We first extract features from our frames using a pre trained convolutional network: we obtain a vector with extracted features, then we cluster in the feature space (K-means)

In [ ]:
import torch 
from torchvision.models import resnet50, ResNet50_Weights
from sklearn.cluster import KMeans
import torchvision.transforms as T
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.decomposition import PCA

In [ ]:
device = "mps"
torch.device(device)
video_array = vid

In [ ]:
weights = ResNet50_Weights.DEFAULT
preprocess = weights.transforms()
model = resnet50(weights= weights)

In [ ]:
# we remove the last layer, don't need it
model = torch.nn.Sequential(*list(model.children())[:-1]).to(device)  # chop off the last FC layer

model.eval()  # important!

In [ ]:
preprocess_custom = T.Compose([
    T.ToTensor(),
    T.Resize((224, 224)),  # OR use T.Resize with aspect ratio preservation + padding
    T.Normalize(mean=[0.485, 0.456, 0.406],  
                std=[0.229, 0.224, 0.225])
])

In [ ]:
# now we extract the feautues:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, frame_array, transform=None, target_transform=None):
        self.frames = frame_array
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return self.frames.shape[0]
    
    def __getitem__(self, idx):
        img = self.frames[idx]  # shape: [H, W]
        
        # Convert grayscale -> RGB by repeating across 3 channels
        img_rgb = np.stack([img] * 3, axis=-1)  # shape: [H, W, 3]
        
        img_pil = Image.fromarray(img_rgb.astype(np.uint8))
        
        if self.transform:
            img = self.transform(img_pil)
        return img

In [ ]:
img_dataset = CustomDataset(video_array, transform=preprocess)

In [ ]:
batch_size = 32
data_loader = DataLoader(img_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# produce the features:
features = []
with torch.no_grad():
    for img in tqdm(data_loader, desc=f'Extracting features '):
        img = img.to(device)
        features.append(model.forward(img))


In [ ]:
features = [img.to('cpu') for img in features]
f = np.concatenate(features)

In [ ]:
f.shape

In [ ]:
features_np = f.mean(axis=(2, 3)) 

In [ ]:
# save the features for other analysis 
save_path_res = Path('/Users/thomasbush/Documents/DSS_Tilburg/data')

np.save(save_path_res / 'features.npy',features_np , allow_pickle=True)


In [ ]:
pca = PCA(n_components = .9)
features_pca = pca.fit_transform(features_np)

### Kmeans

In [ ]:
kmeans = KMeans(n_clusters=500, random_state=42)
kmeans.fit(features_pca)

closest_indices, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, features_pca)
keyframe_indices = sorted(closest_indices)

In [ ]:
from umap import UMAP
import matplotlib.pyplot as plt

reducer = UMAP(n_components=2, random_state=42)
embedding_2d = reducer.fit_transform(features_pca)  # shape: (N, 2)


In [ ]:
labels = kmeans.labels_

plt.figure(figsize=(10, 8))
plt.scatter(embedding_2d[:, 0], embedding_2d[:, 1], c=labels, cmap='tab20', s=5, alpha=0.5, label="All frames")
keyframe_coords = embedding_2d[keyframe_indices]

plt.scatter(keyframe_coords[:, 0], keyframe_coords[:, 1], 
            color='black', marker='*', s=10, label="Keyframes")

plt.legend()
plt.title("Keyframe Selection via K-Means Clustering")
plt.xlabel("UMAP Dim 1")
plt.ylabel("UMAP Dim 2")
plt.tight_layout()
plt.show()

In [ ]:

import matplotlib.cm as cm
# Reduce dimensions for plotting
save_path = Path('/Users/thomasbush/Documents/DSS_Tilburg/data/images')
pca = PCA(n_components=2)
features_pca = pca.fit_transform(features_np)

# Apply KMeans clustering
kmeans = KMeans(n_clusters=500, random_state=42)
kmeans.fit(features_pca)
closest_indices, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, features_pca)

# Assign clusters to all points
labels = kmeans.predict(features_pca)
colors = cm.tab20(labels.astype(float) % 20 / 20)

# Plot
plt.figure(figsize=(12, 9))
plt.scatter(features_pca[:, 0], features_pca[:, 1], c=colors, s=10, alpha=0.5, label="All frames")
plt.scatter(features_pca[closest_indices, 0], features_pca[closest_indices, 1],
            c='black', s=60, marker='*', label="Keyframes", edgecolors='white')

# Annotate a few keyframe indices for draw.io use
for idx in closest_indices[:20]:  # Only first 20 to avoid clutter
    plt.annotate(str(idx), (features_pca[idx, 0], features_pca[idx, 1]), fontsize=8, color='black')

plt.xlabel("PCA Dimension 1")
plt.ylabel("PCA Dimension 2")
plt.title("Keyframe Selection via KMeans Clustering")
plt.legend()
plt.tight_layout()
plt.savefig(save_path / 'pca_kmean_clustering.svg', format="svg", bbox_inches="tight", dpi=300)
plt.show()


In [ ]:
# get some of the frames in question:
# quick script to display 8 keyframes 
idxs = closest_indices[:20]
selected_keyframes = vid[idxs]

# Plot them in a 2x4 grid
fig, axes = plt.subplots(4, 5, figsize=(16, 8))
for i, ax in enumerate(axes.flat):
    ax.imshow(selected_keyframes[i], cmap='gray')
    ax.set_title(f"Keyframe {idxs[i].item()}")
    ax.axis('off')

plt.tight_layout()
plt.savefig(save_path / "selected_keyframes_grid.svg", format='svg', dpi=300) # Save if you want
plt.show()

In [ ]:
# let's als save them:
import os 
os.makedirs(save_path / 'keyframes', exist_ok = True)

for idx in tqdm(closest_indices[:20], desc='generating frames'):

    plt.imsave(save_path / f'keyframes/keyframe{idx}.png', vid[idx],cmap='gray')

### SubModLib

### K-Medoids 

In [ ]:
from sklearn_extra.cluster import KMedoids


In [ ]:
kmedoids = KMedoids(n_clusters=30, metric='euclidean', random_state=42)
kmedoids.fit(features_pca)
closest_indices_kmenoids, _ = pairwise_distances_argmin_min(kmedoids.cluster_centers_, features_pca)
keyframe_indices = sorted(closest_indices_kmenoids)

### DBSCAN 

### Evaluation:

1. Coverage 
2. Redundancy Score
3. Downstream Utility 

### Method 3: Perceptual Hashing

It generates a compact signature of an image based on its visual structure. The algorithm does:
1. resize
2. compute mena pixel intensity 
3. if p>m-> 1 else 0
4. flatten the binary matrix inot a 1D bit string (for img= 8x8 -> 64 flatten)
5. compares two hashes using Hamming distance=The Hamming distance between two equal-length strings of symbols is the number of positions at which the corresponding symbols are different

In [ ]:
dsize = (16, 16)
resized_frames = []

for frame in tqdm(video_array):
    resized = cv2.resize(frame, dsize=dsize, interpolation=cv2.INTER_LINEAR)
    resized_frames.append(resized)

resized_array = np.stack(resized_frames)

In [ ]:
# we compute the mean intensity 
m = resized_array.mean(axis=(0,))
# set to 1 if p > m else 0
binary_img = np.where(resized_array>m, 1, 0)

In [ ]:
# we flatten our array:
flatten_hashes = einops.rearrange(binary_img, 'f h w -> f (h w)')